# 6 - SGD

In [ ]:
#@title Run this cell to download the data and helper files. { display-mode: "form" }
!pip install -U wget
!rm -rf data.zip data lib
!mkdir lib

import wget
wget.download('https://github.com/shengpu1126/BDSI2019-ML/raw/master/lib/config.yaml', 'lib/config.yaml')
wget.download('https://github.com/shengpu1126/BDSI2019-ML/raw/master/lib/helper.py', 'lib/helper.py')
wget.download('https://github.com/shengpu1126/BDSI2019-ML/raw/master/data.zip', 'data.zip')

import zipfile
with zipfile.ZipFile("data.zip","r") as zip_ref:
    zip_ref.extractall(".")

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from lib.helper import load_data, config

In [ ]:
#@title Run this cell to run preprocessing. { display-mode: "form" }

def generate_feature_vector(df):
    """
    Reads a dataframe containing all measurements for a single patient
    within the first 48 hours of the ICU admission, and convert it into
    a feature vector.
    
    Args:
        df: pd.Dataframe, with columns [Time, Variable, Value]
    
    Returns:
        a python dictionary of format {feature_name: feature_value}
        for example, {'Age': 32, 'Gender': 0, 'mean_HR': 84, ...}
    """
    static_variables = config['invariant']
    timeseries_variables = config['timeseries']

    # Replace unknow values
    df = df.replace({-1: np.nan})
    
    # Split time invariant and time series
    static, timeseries = df.iloc[0:5], df.iloc[5:]
    static = static.pivot('Time', 'Variable', 'Value')

    feature_dict = static.iloc[0].to_dict()
    for variable in timeseries_variables:
        measurements = timeseries[timeseries['Variable'] == variable].Value
        feature_dict['mean_' + variable] = np.mean(measurements)
    
    return feature_dict

def impute_missing_values(X):
    """
    For each feature column, impute missing values  (np.nan) with the 
    population mean for that feature.
    
    Args:
        X: np.array, shape (N, d). X could contain missing values
    Returns:
        X: np.array, shape (N, d). X does not contain any missing values
    """
    from sklearn.impute import SimpleImputer
    return SimpleImputer().fit_transform(X)

def normalize_feature_matrix(X):
    """
    For each feature column, normalize all values to range [0, 1].

    Args:
        X: np.array, shape (N, d).
    Returns:
        X: np.array, shape (N, d). Values are normalized per column.
    """
    from sklearn.preprocessing import MinMaxScaler
    return MinMaxScaler().fit_transform(X)

# Load the dataset
# `raw_data` is a dictionary mapping patient ID to the data associated with that patient
raw_data, df_labels = load_data(N=2500)

# Generate features
features = [generate_feature_vector(df) for _, df in tqdm(sorted(raw_data.items()), desc='Generating feature vectors')]
df_features = pd.DataFrame(features).sort_index(axis=1)
feature_names = df_features.columns.tolist()

# Apply imputation and normalization
X, y = df_features.values, df_labels['In-hospital_death'].values
X = impute_missing_values(X)
X = normalize_feature_matrix(X)

# Split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y, random_state=3)
del X, y

For a binary classification task, we have feature vectors $\bar{x} = [x_1, x_2, . . . , x_d]^\intercal \in \mathbb{R}^d$ and labels $y \in \{-1,+1\}$. A classifier is a function mapping from feature vectors to labels $h : \mathbb{R}^d \rightarrow \{-1,+1\}$. For linear classification, the classifiers are thresholded linear mappings:

\begin{equation*}
h(\bar{x};\bar\theta) = \mathrm{sign}(\bar\theta \cdot \bar x) = \Big\{\begin{matrix} 
+1 & \bar\theta \cdot \bar x > 0\\ 
-1 & \bar\theta \cdot \bar x \leq 0
\end{matrix}
\end{equation*}


We have also defined the following:

- Training error
\begin{equation}
    \mathcal{E}_n (\bar{\theta}) = \frac{1}{N} \sum_{i=1}^N [[y^{(i)} \neq h(\bar{x}^{(i)};\bar{\theta})]]
    \nonumber
\end{equation}

- Empirical risk (wrt some loss function)
\begin{equation}
    R_n(\bar{\theta}) = \frac{1}{N} \sum_{i=1}^N \mathrm{loss}(y \; (\bar{\theta} \cdot \bar{x}^{(i)}))
    \nonumber
\end{equation}

So far we have seen 3 loss functions suitable for a (binary) classification task with label $y\in\{-1,+1\}$:
- $\mathrm{loss}_{0/1}(z) = [[z\leq 0]]$
- $\mathrm{loss}_{h}(z) = \max\{0, 1-z\}$
- $\mathrm{loss}_{\log}(z) = \log_2(1+e^{-z})$

Here $z = y \; (\bar{\theta} \cdot \bar{x})$; Interpretation for $z$: "a numerical quantity that indicates whether or not you're right, and to what extent you are right or wrong". 

## Review our data...

In [ ]:
sel = [0,1,2,3,6,8,55,88]    # Select an easy subset of patients
X = X_train[sel][:, [0,20]]  # Select age and mean_HR
y = y_train[sel]

In [ ]:
def plot_data(X, y, markersize=25):
    fig = plt.figure(figsize=(5,5))
    for xi, yi in zip(X, y):
        if yi == -1:
            plt.scatter(xi[0], xi[1], c='r', marker='o', s=markersize)
        elif yi == 1:
            plt.scatter(xi[0], xi[1], c='g', marker='x', s=markersize)
    plt.axis('equal')
    plt.xlabel('Age')
    plt.ylabel('mean_HR')
    plt.xlim(0,1)
    plt.ylim(0,1)
    plt.grid(True)
    return fig

In [ ]:
fig = plot_data(X, y)
plt.show()

## (1) SGD with hinge loss

The empirical risk with hinge loss is defined as:
\begin{equation}
    J(\bar{\theta}) = \frac{1}{N} \sum_{i=1}^N \max\{0, 1-y \; (\bar{\theta} \cdot \bar{x}^{(i)})\}
    \nonumber
\end{equation}

Calculate the gradient $\nabla_{\bar\theta}J(\bar{\theta})$:

>$\nabla_{\bar\theta}J(\bar{\theta}) = \dots$

Write down the update rule using SGD upon seeing one example $(\bar{x}^{(i)}, y^{(i)})$: 

>$\bar\theta^{(k+1)} = \dots$

In [ ]:
# Implement SGD with hinge loss
def SGD_hinge(X, y, lr=1e-3, max_iters=1e4):
    n, d = X.shape
    theta = np.zeros(d)
    losses = []
    
    n_iter = 0
    while n_iter < max_iters:
        # Loop through the entire dataset
        for xi, yi in zip(X, y):
            n_iter += 1
            
            # Update theta based on each example (xi, yi)
            theta = ??
            
            # Track training progress by calculating empirical risk after each update
            losses.append(empirical_risk(X, y, theta))
        
        # Optionally, shuffle the data points
        X, y = ??shuffle(X, y)??
    
    return theta, losses

In [ ]:
X_ = np.concatenate([np.ones((X.shape[0],1)), X], axis=1) # Create a synthetic feature dimension to incorporate the offset term
theta, losses = SGD_hinge(X_, y, 1e-2, 1e4)
print(theta)

In [ ]:
# Visualize training progress
plt.plot(losses)
plt.xlabel('iterations')
plt.ylabel('Loss')
plt.show()

In [ ]:
# Visualize the classification boundary in the same plot. Does it match your intuition?
x_clf = np.linspace(0,1,100)
y_clf = (theta_sgd[0] * x_clf + theta_sgd[2]) / -theta_sgd[1]
fig = plot_data(X, y)
plt.plot(x_clf, y_clf, 'k-')
plt.show()

## (2) SGD with hinge loss and L2-regularization

The empirical risk with hinge loss and L2-regularization is defined as:
\begin{equation}
    J(\bar{\theta}) = \frac{\lambda}{2}\|\bar\theta\|_2^2 + \frac{1}{N} \sum_{i=1}^N \max\{0, 1-y \; (\bar{\theta} \cdot \bar{x}^{(i)})\}
    \nonumber
\end{equation}

Calculate the gradient $\nabla_{\bar\theta}J(\bar{\theta})$:

>$\nabla_{\bar\theta}J(\bar{\theta}) = \dots$

Write down the update rule using SGD upon seeing one example $(\bar{x}^{(i)}, y^{(i)})$: 

>$\bar\theta^{(k+1)} = \dots$

In [ ]:
def SGD_hinge_L2(X, y, lr=1e-3, lambda_=1):
    theta = None
    losses = []
    return theta, losses

In [ ]:
X_ = np.concatenate([X, np.ones((X.shape[0],1))], axis=1)
theta, losses = SGD_hinge(X_, y, 1e-2)

In [ ]:
# Visualizations...

## (3) Extensions
- Experiment with different learning rates and/or regularization strengths.
- What would the update rule(s) be if we do not regularize the offset parameter?

## (4) Extra: SGD with logistic loss

\begin{equation}
J(\bar\theta) = \frac{1}{N} \sum_{i=1}^{N} \ln(1+\exp^{-y^{(i)}\bar{\theta}\cdot \bar{x}^{(i)}})
\end{equation}

Calculate the gradient $\nabla_{\bar\theta}J(\bar{\theta})$:

>$\nabla_{\bar\theta}J(\bar{\theta}) = \dots$

Write down the update rule using SGD upon seeing one example $(\bar{x}^{(i)}, y^{(i)})$: 

>$\bar\theta^{(k+1)} = \dots$

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
def SGD_logistic(X, y, lr=1e-3):
    theta = None
    losses = []
    return theta